In [11]:
import os

os.environ['SPARK_HOME'] = "/Applications/Spark"
os.environ['SPARK_LOCAL_IP'] = '127.0.0.1'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python'

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit

In [13]:
spark: SparkSession = SparkSession.builder.appName("Wine quality").getOrCreate()

In [14]:
red_wine_df = spark.read.csv("./data/winequality-red.csv", header=True, inferSchema=True, sep=";")
white_wine_df = spark.read.csv("./data/winequality-white.csv", header=True, inferSchema=True, sep=";")
df = red_wine_df.withColumn("color", lit("red")).union(white_wine_df.withColumn("color", lit("white")))
df.createOrReplaceTempView("wines")

In [15]:
numeric_cols = [col for col in df.columns if col not in ("color", "quality")]

{
   col_name: df.stat.corr(col_name, "quality")
   for col_name in numeric_cols
}



{'fixed_acidity': -0.0767432079096202,
 'volatile_acidity': -0.26569947761146706,
 'citric_acid': 0.08553171718367798,
 'residual_sugar': -0.036980484585769774,
 'chlorides': -0.2006655004351014,
 'free_sulfur_dioxide': 0.055463058616632414,
 'total_sulfur_dioxide': -0.04138545385560937,
 'density': -0.3058579060694186,
 'pH': 0.01950570371443557,
 'sulphates': 0.03848544587651391,
 'alcohol': 0.4443185200076535}

In [16]:
df.sparkSession.sql("SELECT color, COUNT(*) FROM wines GROUP BY color;").show()

+-----+--------+
|color|count(1)|
+-----+--------+
|  red|    1599|
|white|    4898|
+-----+--------+



In [17]:
df_single_partition = df.coalesce(1)
df_single_partition.write.mode("overwrite").option("header", "true").csv("./data/winequality-all.csv")